## Use taxonomic read classifications from MEGAN6 to extract Phylum-specific FastQs from Arthropoda and Alveolata

In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Mon Mar 30 08:52:39 PDT 2020
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 16.04.6 LTS
Release:	16.04
Codename:	xenial

------------
HOSTNAME: 
swoose

------------
Computer Specs:

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                24
On-line CPU(s) list:   0-23
Thread(s) per core:    2
Core(s) per socket:    6
Socket(s):             2
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 44
Model name:            Intel(R) Xeon(R) CPU           X5670  @ 2.93GHz
Stepping:              2
CPU MHz:               2925.931
BogoMIPS:              5851.96
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              12288K
NUMA node0 CPU(s):     0-23
Flags:                 fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr

No LSB modules are available.


### Set variables

In [2]:
# Set data directories
%env crab_data=/home/sam/data/C_bairdi/RNAseq
%env hemat_data=/home/sam/data/Hematodinium/RNAseq
%env wd=/home/sam/analyses

# Programs
%env seqtk=/home/sam/programs/seqtk-1.3/seqtk

env: crab_data=/home/sam/data/C_bairdi/RNAseq
env: hemat_data=/home/sam/data/Hematodinium/RNAseq
env: wd=/home/sam/analyses
env: seqtk=/home/sam/programs/seqtk-1.3/seqtk


#### Input data are here:

FastAs: 

- https://gannet.fish.washington.edu/Atumefaciens/20200114_cbai_MEGAN_read_extractions/

- https://gannet.fish.washington.edu/Atumefaciens/20200323_cbai_MEGAN_read_extractions/

Trimmed-FastQs:

- https://gannet.fish.washington.edu/Atumefaciens/20191218_cbai_fastp_RNAseq_trimming/

- https://gannet.fish.washington.edu/Atumefaciens/20200318_cbai_RNAseq_fastp_trimming/

### Extract phylum-specific reads from trimmed FastQ files


Use FastA IDs from MEGAN6 taxonomic read extraction FastAs to pull out appropriate reads from each phylum (Arthropoda and Alveolata). This is performed because MEGAN6 strips paired read ID after the first space. As such, the resulting read extractions using MEGAN end up with a FastA file containing two reads with identicial headers. Not sure if this will cause any downstream issues (i.e. with Trinity) where paired end data is used, so playing it safe and using the truncated IDs to pull FastQs with complete sequence headers for use in subsequent data wrangling.

In [3]:
%%bash

timestamp=$(date +%Y%m%d)


for directory in ${crab_data} ${hemat_data}
do
	# Get species name
	species=$(echo ${directory} | awk -F"/" '{print $5}')
    
    # Make new directory and change to that directory ("$_" means use previous command's argument)
    mkdir --parents ${wd}/"${timestamp}"."${species}"_megan_reads \
    && cd "$_" || exit

	# Set seqtk list filename
	seqtk_list=${timestamp}.${species}.seqtk.read_id.list

	# Set output FastQ filenames
	R1_fq=${timestamp}.${species}.megan_R1.fq
	R2_fq=${timestamp}.${species}.megan_R2.fq

	######################################################
	# Create FastA IDs list to use for sequence extraction
	######################################################
	for fasta in "${directory}"/*.fasta
	do
      echo "Pulling FastA IDs from ${fasta}"
      echo ""
      grep ">" "${fasta}" | awk 'sub(/^>/, "")'
	done | sort -u >> "${seqtk_list}"
    
    
    echo ""
    echo "Finished with FastA ID extraction."
    echo ""
    echo "Moving on to read extractions..." 
    echo ""
    echo ""
    
    
    ######################################################
	# Extract corresponding R1 and R2 reads using seqtk FastA ID list
    ######################################################
	for fastq in "${directory}"/*R1*.gz
	do
      echo "Extracting R1 reads from ${fastq}"
      echo ""
	  ${seqtk} subseq "${fastq}" "${seqtk_list}" >> "${R1_fq}"
	done
    
    echo ""
    echo "Done with R1 read extractions"
    echo ""
    echo "------------"
    echo ""
    echo "Extracting R2 reads from ${fastq}"

	for fastq in "${directory}"/*R2*.gz
	do
      echo "Extracting R2 reads from ${fastq}"
      echo ""
	  ${seqtk} subseq "${fastq}" "${seqtk_list}" >> "${R2_fq}"
	done
    
    echo "-------------------------------------"
    # Print working directoyr and list files
    pwd
	ls -ltrh
    echo ""
    echo "-------------------------------------"
    echo ""
done



Finished with FastA ID extraction.

Moving on to read extractions...


Extracting R1 reads from /home/sam/data/C_bairdi/RNAseq/113_R1_001.fastp-trim.202003184225.fq.gz

Extracting R1 reads from /home/sam/data/C_bairdi/RNAseq/118_R1_001.fastp-trim.202003184931.fq.gz

Extracting R1 reads from /home/sam/data/C_bairdi/RNAseq/127_R1_001.fastp-trim.202003185538.fq.gz

Extracting R1 reads from /home/sam/data/C_bairdi/RNAseq/132_R1_001.fastp-trim.202003180140.fq.gz

Extracting R1 reads from /home/sam/data/C_bairdi/RNAseq/151_R1_001.fastp-trim.202003180619.fq.gz

Extracting R1 reads from /home/sam/data/C_bairdi/RNAseq/173_R1_001.fastp-trim.202003181159.fq.gz

Extracting R1 reads from /home/sam/data/C_bairdi/RNAseq/178_R1_001.fastp-trim.202003181815.fq.gz

Extracting R1 reads from /home/sam/data/C_bairdi/RNAseq/221_R1_001.fastp-trim.202003182309.fq.gz

Extracting R1 reads from /home/sam/data/C_bairdi/RNAseq/222_R1_001.fastp-trim.202003182800.fq.gz

Extracting R1 reads from /home/sam/data/C_bair

### See if resulting FastQ files have same number of lines (i.e. same number of reads)

In [5]:
%%bash
for directory in /home/sam/analyses/20200330.C_bairdi_megan_reads /home/sam/analyses/20200330.Hematodinium_megan_reads
do
  cd "${directory}" || exit
  for file in *
  do
    wc -l ${file}
  done
done

295485796 20200330.C_bairdi.megan_R1.fq
295485796 20200330.C_bairdi.megan_R2.fq
73871479 20200330.C_bairdi.seqtk.read_id.list
7686516 20200330.Hematodinium.megan_R1.fq
7686516 20200330.Hematodinium.megan_R2.fq
1921637 20200330.Hematodinium.seqtk.read_id.list
